In [ ]:
#실습 목표 - ML 모델들의 보안 취약점에 대한 인식을 높이고, 요즘 화두가 되고있는 적대적 머신 러닝에 대한 통찰력을 제공
#실습 개요 - 가장 많이 사용되는 공격 방법 중 하나인 FGSM (Fast Gradient Sign Attack)을 이용해 MNIST 분류기를 속이기



In [ ]:
'''
위협모델 :
- 가장 중요한 목표는 입력 데이터에 최소한의 작은 변화를 추가하여 이것이 의도적으로 잘못 분류되게 하는 것
- 정보에 대한 가정에는 여러 종류가 있는데, 보통 화이트박스 와 블랙박스 두 가지가 있다.
  - 화이트박스 공격은 공격자가 모델에 대해 아키텍처, 입력, 출력, 가중치를 포함한 모든 것을 알고 있고 접근할 수 있다고 가정합니다.
  - 블랙박스 공격은 공격자가 모델의 입력과 출력에 대해서만 접근 가능하고 모델의 가중치와 아키텍처에 관한 내용은 모른다고 가정합니다.
- 소스/타겟 오분류 는 공격자가 원래 특정 소스 클래스의 이미지를 다른 특정 대상 클래스로 분류하도록 변경하려고 함을 의미함
'''

In [ ]:
'''
빠른 변화도 부호 공격 : 
- 공격은 입력 데이터에서 계산된 손실 변화도를 사용하고 입력 데이터를 조정하여 손실이 최대가 되게 합니다.
'''

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

# NOTE: 아래는 MNIST 데이터셋을 내려받을 때 "User-agent" 관련한 제한을 푸는 코드입니다.
#       더 자세한 내용은 https://github.com/pytorch/vision/issues/3497 을 참고해주세요.
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [ ]:
입력
이 학습서에는 입력이 3 개이며 다음과 같이 정의됩니다:

epsilons - 실행에 사용할 엡실론의 리스트입니다. 엡실론 0의 값은 원래 테스트 셋의 모델 성능을 나타내므로 목록에 유지하는 것이 중요합니다. 또한 직관적으로 엡실론이 클수록 작은 변화가 더 눈에 띄지만 모델 정확도를 저하 시키는 측면에서 더 효과가 있습니다. 여기서 데이터의 범위는 0-1 이기 때문에 엡실론의 값은 1을 초과할 수 없습니다.

pretrained_model - pytorch/examples/mnist
를 통해 미리 학습된 MNIST 모델의 경로. 튜토리얼을 간편하게 하려면 여기 에서 미리 학습된 모델을 다운로드하세요.

use_cuda - CUDA 를 사용할지 말지 정하는 이진 플래그. 본 튜토리얼에서는 CPU 시간이 오래 걸리지 않으므로 CUDA를 지원하는 GPU 의 여부는 중요하지 않습니다.

In [ ]:
epsilons = [0, .05, .1, .15, .2, .25, .3]
pretrained_model = "data/lenet_mnist_model.pth"
use_cuda=True

In [ ]:

공격을 받는 모델
앞서 말한대로, 공격받는 모델은 pytorch/examples/mnist 와 동일한 MNIST 모델입니다. 본인의 MNIST 모델을 학습 및 저장하는 방식으로 하거나 
제공된 모델을 다운로드 해 사용하는 식으로 진행할 수 있습니다. 여기서 Net 정의 및 테스트 데이터 로더는 MNIST 예제에서 복사하였습니다. 
이 섹션의 목적은 모델과 데이터 로더를 정의한 다음, 모델을 초기화하고 미리 학습된 가중치를 읽어오는 것입니다.


In [4]:
# LeNet 모델 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# MNIST 테스트 데이터셋과 데이터로더 선언
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            ])),
        batch_size=1, shuffle=True)

# 어떤 디바이스를 사용할지 정의
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

# 모델 초기화하기
model = Net().to(device)

# 미리 학습된 모델 읽어오기
model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))

# 모델을 평가 모드로 설정하기. 드롭아웃 레이어들을 위해 사용됨
model.eval()

CUDA Available:  False


FileNotFoundError: ignored

In [ ]:
FGSM 공격
이제 원래 입력을 교란시켜 적대적인 예를 만드는 함수를 정의 할 수 있습니다. fgsm_attack 함수는 입력 파라미터로 3가지를 가집니다. 첫 번째는 원본 이미지 ( xx ), 두 번째는 엡실론 으로 픽셀 단위의 작은 변화를 주는 값입니다 ( \epsilonϵ ). 마지막은 data_grad 로 입력 영상 ( \nabla_{x} J(\mathbf{\theta}, \mathbf{x}, y)∇ 
x
​
 J(θ,x,y) ) 에 대한 변화도 손실 값입니다. 아래 식에 따른 작은 변화가 적용된 이미지를 생성합니다.

perturbed\_image = image + epsilon*sign(data\_grad) = x + \epsilon * sign(\nabla_{x} J(\mathbf{\theta}, \mathbf{x}, y))
perturbed_image=image+epsilon∗sign(data_grad)=x+ϵ∗sign(∇ 
x
​
 J(θ,x,y))
마지막으로 데이터의 원래 범위를 유지하기 위해, 작은 변화가 적용된 이미지가 [0,1][0,1] 범위로 잘립니다.

In [ ]:
# FGSM 공격 코드
def fgsm_attack(image, epsilon, data_grad):
    # data_grad 의 요소별 부호 값을 얻어옵니다
    sign_data_grad = data_grad.sign()
    # 입력 이미지의 각 픽셀에 sign_data_grad 를 적용해 작은 변화가 적용된 이미지를 생성합니다
    perturbed_image = image + epsilon*sign_data_grad
    # 값 범위를 [0,1]로 유지하기 위해 자르기(clipping)를 추가합니다
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # 작은 변화가 적용된 이미지를 리턴합니다
    return perturbed_image